In [ ]:
import yfinance as yf
from arch import arch_model
from arch.__future__ import reindexing
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
ticker= "aapl"
stock_data = yf.download(ticker, start="2016–01–04", end="2021–04–30")
stock_data['Return'] = 100 * (stock_data['Close'].pct_change())
stock_data.dropna(inplace=True)

In [ ]:
fig = plt.figure()
fig.set_figwidth(12)
plt.plot(stock_data['Return'], label='Daily Returns')
plt.legend(loc='upper right')
plt.title('Daily Returns Over Time')
plt.show()

In [ ]:
daily_volatility = stock_data['Return'].std()
print('Daily volatility: ', '{:.2f}%'.format(daily_volatility))
monthly_volatility = math.sqrt(21) * daily_volatility
print('Monthly volatility: ', '{:.2f}%'.format(monthly_volatility))
annual_volatility = math.sqrt(252) * daily_volatility
print('Annual volatility: ', '{:.2f}%'.format(annual_volatility))

In [ ]:
garch_model = arch_model(stock_data['Return'], p=1, q=1,
 mean='constant', vol='GARCH', dist='normal')
gm_result = garch_model.fit(disp='off')
print(gm_result.params)

In [ ]:
print('\n')
gm_forecast = gm_result.forecast(horizon=5)
print(gm_forecast.variance[-1:])

In [ ]:
rolling_predictions = []
test_size = 365
for i in range(test_size):
 train = stock_data['Return'][:-(test_size-i)]
 model = arch_model(train, p=1, q=1)
 model_fit = model.fit(disp='off')
 pred = model_fit.forecast(horizon=1)
 rolling_predictions.append(np.sqrt(pred.variance.values[-1,:][0]))
rolling_predictions = pd.Series(rolling_predictions, index=stock_data['Return'].index[-365:])

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(rolling_predictions)
plt.title('Rolling Prediction')
plt.show()
plt.figure(figsize=(12,4))
plt.plot(stock_data['Return'][-365:])
plt.plot(rolling_predictions)
plt.title('Volatility Prediction - Rolling Forecast')
plt.legend(['True Daily Returns', 'Predicted Volatility'])
plt.show()